In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import xgboost as xgb

In [27]:
data = pd.read_csv('D:/备份/0123_busno数据.csv',low_memory=False)
data = data[data['busno'] != 999]
data['enstation_time'] = pd.to_datetime(data['enstation_time'])
data['day'] = data['enstation_time'].dt.day
data['hour'] = data['enstation_time'].dt.hour
data['minute'] = data['enstation_time'].dt.minute
# data['second'] = data['enstation_time'].dt.second
data = data[data['enstation_name'] != data['exstation_name']]
data.dropna(subset='card_type',inplace=True)
data = data[data['card_no'] != 999]
# data.drop(['enstation_street','exstation_street'],axis=1,inplace=True)
data.dropna(subset='card_type',inplace=True)
data['card_type'] = data['card_type'].astype('int')

In [28]:
data['exstation_district'] = data['exstation_street'].str[:3]
data['enstation_district'] = data['enstation_street'].str[:3]

In [29]:
data = data[(data['exstation_district'] == '武侯区') | (data['exstation_district'] == '锦江区') |
     (data['exstation_district'] == '青羊区') | (data['exstation_district'] == '金牛区') |
     (data['exstation_district'] == '成华区')].reset_index(drop=True)
data = data[(data['enstation_district'] == '武侯区') | (data['enstation_district'] == '锦江区') |
     (data['enstation_district'] == '青羊区') | (data['enstation_district'] == '金牛区') |
     (data['enstation_district'] == '成华区')].reset_index(drop=True)

In [6]:
line_names = data['line_name'].unique()

In [7]:
data

,card_no,enstation_time,enstation_name,orientation,line_name,lon1,lat1,exstation_name,lon2,lat2,...,exstation_street,bus_lenth,station_num,card_type,busno,day,hour,minute,exstation_district,enstation_district
0,1.000000e+15,2022-11-03 18:12:15,新鸿路,2.0,71,104.1072,30.66170,小龙桥路,104.110880,30.659771,...,成华区新鸿路街道,32,16.0,69,21042442.0,3,18,12,成华区,成华区
1,1.000000e+15,2022-11-03 07:13:01,文家大道南,1.0,232,103.9421,30.69342,文家大道成温路口,103.940245,30.689269,...,青羊区文家街道,16,9.0,69,21025017.0,3,7,13,青羊区,青羊区
2,1.000000e+15,2022-11-03 08:02:53,凯德风尚,1.0,309,103.9575,30.66886,光华大道一段东二站,103.952034,30.668457,...,青羊区文家街道,42,8.0,69,21143292.0,3,8,2,青羊区,青羊区
3,1.000000e+15,2022-11-03 17:43:10,西御街,1.0,13,104.0627,30.65633,清波社区,103.986536,30.681534,...,青羊区苏坡街道,26,24.0,69,21022425.0,3,17,43,青羊区,青羊区
4,1.000000e+15,2022-11-03 07:34:37,清波社区,2.0,228,103.9853,30.68277,羊犀立交南,104.003737,30.698363,...,金牛区金泉街道,22,8.0,69,21022181.0,3,7,34,金牛区,青羊区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314851,9.963942e+15,2022-11-15 08:40:43,新成彭路口,2.0,25,104.0473,30.72058,川陕路口,104.098562,30.707368,...,成华区双水碾街道,30,11.0,69,21042275.0,15,8,40,成华区,金牛区
6314852,9.986589e+15,2022-11-15 12:42:37,上沙河铺路,2.0,47,104.1229,30.62358,地铁东大路站,104.111689,30.633912,...,锦江区双桂路街道,39,7.0,69,21012416.0,15,12,42,锦江区,锦江区
6314853,9.994650e+15,2022-11-15 09:39:54,二环成仁公交站,2.0,120,104.0981,30.62448,地铁槐树店站,104.138256,30.648232,...,成华区万年场街道,29,14.0,69,21012078.0,15,9,39,成华区,锦江区
6314854,3.105170e+18,2022-11-15 08:26:48,致强环街,1.0,71,104.1159,30.70092,二仙桥西路,104.125499,30.681027,...,成华区桃蹊路街道,32,5.0,32,21042502.0,15,8,26,成华区,成华区


In [13]:
for line_name in tqdm(line_names[15:]):
    try:
        data_tem = data[data['line_name'] == line_name].reset_index(drop=True)
        ex_station_en = LabelEncoder()
        y_data = ex_station_en.fit_transform(data_tem['exstation_name'])
        x_data = data_tem.drop(['enstation_time','enstation_name','lon2','lat2','exstation_name','line_name','exstation_street',
                                'enstation_street','exstation_district','enstation_district'],axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,random_state=528)
        train_data = xgb.DMatrix(X_train, label=y_train)
        test_data = xgb.DMatrix(X_test,label=y_test)
        params = {
            'objective': 'multi:softmax',
            'num_class': data_tem['exstation_name'].nunique(),
            'eval_metric': 'merror',
            'device': 'cuda',
#             'learning_rate':0.05
        }
        model = xgb.train(params=params, dtrain=train_data, num_boost_round=5000,
                          early_stopping_rounds=50,evals=[(train_data,'train'),(test_data,'eval')],verbose_eval=0)
        model.save_model('0401_XG5/' + line_name + '路模型.json')

    except Exception as e:
        print(f'{line_name}路模型训练失败')
        print(e)

 99%|█████████████████████████████████████████████████████████████████████████████▏| 490/495 [2:05:14<00:05,  1.13s/it]

G50路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


100%|█████████████████████████████████████████████████████████████████████████████▊| 494/495 [2:05:17<00:00,  1.22it/s]

D705路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


100%|██████████████████████████████████████████████████████████████████████████████| 495/495 [2:05:17<00:00, 15.19s/it]

G166路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [14]:
from sklearn.metrics import classification_report
import numpy as np

In [15]:
data

,card_no,enstation_time,enstation_name,orientation,line_name,lon1,lat1,exstation_name,lon2,lat2,...,exstation_street,bus_lenth,station_num,card_type,busno,day,hour,minute,exstation_district,enstation_district
0,1.000000e+15,2022-11-03 18:12:15,新鸿路,2.0,71,104.1072,30.66170,小龙桥路,104.110880,30.659771,...,成华区新鸿路街道,32,16.0,69,21042442.0,3,18,12,成华区,成华区
1,1.000000e+15,2022-11-03 07:13:01,文家大道南,1.0,232,103.9421,30.69342,文家大道成温路口,103.940245,30.689269,...,青羊区文家街道,16,9.0,69,21025017.0,3,7,13,青羊区,青羊区
2,1.000000e+15,2022-11-03 08:02:53,凯德风尚,1.0,309,103.9575,30.66886,光华大道一段东二站,103.952034,30.668457,...,青羊区文家街道,42,8.0,69,21143292.0,3,8,2,青羊区,青羊区
3,1.000000e+15,2022-11-03 17:43:10,西御街,1.0,13,104.0627,30.65633,清波社区,103.986536,30.681534,...,青羊区苏坡街道,26,24.0,69,21022425.0,3,17,43,青羊区,青羊区
4,1.000000e+15,2022-11-03 07:34:37,清波社区,2.0,228,103.9853,30.68277,羊犀立交南,104.003737,30.698363,...,金牛区金泉街道,22,8.0,69,21022181.0,3,7,34,金牛区,青羊区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314851,9.963942e+15,2022-11-15 08:40:43,新成彭路口,2.0,25,104.0473,30.72058,川陕路口,104.098562,30.707368,...,成华区双水碾街道,30,11.0,69,21042275.0,15,8,40,成华区,金牛区
6314852,9.986589e+15,2022-11-15 12:42:37,上沙河铺路,2.0,47,104.1229,30.62358,地铁东大路站,104.111689,30.633912,...,锦江区双桂路街道,39,7.0,69,21012416.0,15,12,42,锦江区,锦江区
6314853,9.994650e+15,2022-11-15 09:39:54,二环成仁公交站,2.0,120,104.0981,30.62448,地铁槐树店站,104.138256,30.648232,...,成华区万年场街道,29,14.0,69,21012078.0,15,9,39,成华区,锦江区
6314854,3.105170e+18,2022-11-15 08:26:48,致强环街,1.0,71,104.1159,30.70092,二仙桥西路,104.125499,30.681027,...,成华区桃蹊路街道,32,5.0,32,21042502.0,15,8,26,成华区,成华区


In [20]:
accuracy = np.empty(0)
precision = np.empty(0)
recall = np.empty(0)
f1 = np.empty(0)
ln = np.empty(0)
for line_name in tqdm(line_names):
    try: 
        data_tem = data[data['line_name'] == line_name].reset_index(drop=True)
        ex_station_en = LabelEncoder()
        y_data = ex_station_en.fit_transform(data_tem['exstation_name'])
        x_data = data_tem.drop(['enstation_time','enstation_name','lon2','lat2','exstation_name','line_name','exstation_street',
                                'enstation_street','exstation_district','enstation_district'],axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.2,random_state=528)
        train_data = xgb.DMatrix(X_train, label=y_train)
        test_data = xgb.DMatrix(X_test,label=y_test)

        model = xgb.Booster()
        model.load_model('0401_XG5/' + line_name + '路模型.json')
        y_pre = model.predict(test_data)

        report_dict = classification_report(y_test,y_pre,output_dict=True)
        report_df = pd.DataFrame(report_dict).transpose()
        accuracy = np.append(accuracy,report_df.iloc[-3]['support'])
        precision = np.append(precision,report_df.iloc[-1]['precision'])
        recall = np.append(recall,report_df.iloc[-1]['recall'])
        f1 = np.append(f1,report_df.iloc[-1]['f1-score'])
        ln = np.append(ln,line_name)
    except Exception as e:
        print(f'{line_name}路模型训练失败')
        print(e)
    
out = pd.DataFrame({'line_name':ln,'accuracy':accuracy,'precision':precision,'reall':recall,'f1':f1})

  0%|▏                                                                               | 1/510 [00:07<1:05:41,  7.74s/it]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

 15%|███████████▉                                                                     | 75/510 [06:43<48:31,  6.69s/it]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

 25%|████████████████████▍                                                           | 130/510 [12:09<19:05,  3.02s/it]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

 35%|████████████████████████████▏                                                   | 180/510 [14:52<26:12,  4.77s/it]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

 44%|███████████████████████████████████▎                                            | 225/510 [16:56<16:00,  3.37s/it]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

...

G50路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\

D705路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


100%|████████████████████████████████████████████████████████████████████████████████| 510/510 [22:58<00:00,  2.70s/it]

G166路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [21]:
for keys in out.columns[1:]:
    print(f'{keys}: {np.average(out[keys])}')

accuracy: 0.4403169995536359
precision: 0.43050194385287144
reall: 0.4403169995536359
f1: 0.4260962764283004


# 预测

In [25]:
import os

In [22]:
df = pd.read_csv('D:/备份/0226UTC.csv',low_memory=False)

In [30]:
df['enstation_time'] = pd.to_datetime(df['enstation_time'])

In [26]:
file_names = os.listdir('0401_XG5/')
line_names = [file_name.replace("路模型.json",'') for file_name in file_names]

In [37]:
df_tem_TC[['card_no', 'orientation','line_name',
               'lon1', 'lat1','bus_lenth', 'station_num', 'card_type','busno' ,'day', 'hour', 'minute']]

,card_no,orientation,line_name,lon1,lat1,bus_lenth,station_num,card_type,busno,day,hour,minute
0,6.100000e+15,1.0,1001,104.1209,30.66063,14,0.0,15,21012192.0,3,9,12
1,6.100000e+15,1.0,1001,104.1190,30.65801,14,1.0,15,21012005.0,3,13,57
2,6.100000e+15,1.0,1001,104.1187,30.66146,14,12.0,15,21012192.0,3,8,19
3,6.100000e+15,1.0,1001,104.1142,30.65730,14,4.0,15,21012005.0,3,17,49
4,6.100000e+15,1.0,1001,104.1199,30.65692,14,1.0,15,21012005.0,3,16,38
...,...,...,...,...,...,...,...,...,...,...,...,...
10532,6.100000e+15,1.0,1001,104.1142,30.65723,14,4.0,6,21015016.0,15,10,15
10533,6.100000e+15,1.0,1001,104.1161,30.65340,14,3.0,15,21012005.0,15,8,3
10534,6.100100e+15,1.0,1001,104.1191,30.65805,14,1.0,15,21012192.0,15,8,52
10535,6.100100e+15,1.0,1001,104.1063,30.66095,14,7.0,6,21012192.0,15,9,6


In [38]:
data.drop(['enstation_time','enstation_name','lon2','lat2','exstation_name','line_name','exstation_street',
                                'enstation_street','exstation_district','enstation_district'],axis=1)

,card_no,orientation,lon1,lat1,bus_lenth,station_num,card_type,busno,day,hour,minute
0,1.000000e+15,2.0,104.1072,30.66170,32,16.0,69,21042442.0,3,18,12
1,1.000000e+15,1.0,103.9421,30.69342,16,9.0,69,21025017.0,3,7,13
2,1.000000e+15,1.0,103.9575,30.66886,42,8.0,69,21143292.0,3,8,2
3,1.000000e+15,1.0,104.0627,30.65633,26,24.0,69,21022425.0,3,17,43
4,1.000000e+15,2.0,103.9853,30.68277,22,8.0,69,21022181.0,3,7,34
...,...,...,...,...,...,...,...,...,...,...,...
6314851,9.963942e+15,2.0,104.0473,30.72058,30,11.0,69,21042275.0,15,8,40
6314852,9.986589e+15,2.0,104.1229,30.62358,39,7.0,69,21012416.0,15,12,42
6314853,9.994650e+15,2.0,104.0981,30.62448,29,14.0,69,21012078.0,15,9,39
6314854,3.105170e+18,1.0,104.1159,30.70092,32,5.0,32,21042502.0,15,8,26


In [42]:
out = []
for line_name in tqdm(line_names):
    df_tem_TC = data[data['line_name'] == f"{line_name}"].reset_index(drop=True)
    ex_station_en = LabelEncoder()
    ex_station_en.fit(df_tem_TC['exstation_name'])
    df_tem_UTC = df[df['line_name'] == f'{line_name}']
    x_data = df_tem_UTC[['card_no','orientation','lon1','lat1','bus_lenth','station_num',
                         'card_type','busno','day','hour','minute']]
    x_data = xgb.DMatrix(x_data)
    model = xgb.Booster()
    model.load_model('0401_XG5/' + line_name + '路模型.json')
    y_data = model.predict(x_data)
    df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
    out.append(df_tem_UTC)

  0%|                                                                                          | 0/507 [00:00<?, ?it/s]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
  0%|▏                                                                                 | 1/507 [00:02<21:33,  2.56s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

  3%|██▍                                                                              | 15/507 [00:38<21:46,  2.66s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
  3%|██▌                                                                              | 16/507 [00:41<20:49,  2.55s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

  6%|████▊                                                                            | 30/507 [01:19<22:38,  2.85s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
  6%|████▉                                                                            | 31/507 [01:22<22:24,  2.83s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

  9%|███████▏                                                                         | 45/507 [01:54<17:15,  2.24s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
  9%|███████▎                                                                         | 46/507 [01:56<17:10,  2.24s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

 12%|█████████▌                                                                       | 60/507 [02:36<20:32,  2.76s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
 12%|█████████▋                                                                       | 61/507 [02:38<20:01,  2.69s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

...

 84%|███████████████████████████████████████████████████████████████████▏            | 426/507 [28:12<02:37,  1.94s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
 84%|███████████████████████████████████████████████████████████████████▍            | 427/507 [28:13<02:22,  1.79s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

 88%|██████████████████████████████████████████████████████████████████████          | 444/507 [28:38<01:38,  1.56s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
 88%|██████████████████████████████████████████████████████████████████████▏         | 445/507 [28:39<01:35,  1.55s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

 91%|████████████████████████████████████████████████████████████████████████▍       | 459/507 [29:03<01:25,  1.78s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
 91%|████████████████████████████████████████████████████████████████████████▌       | 460/507 [29:06<01:33,  2.00s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

 94%|███████████████████████████████████████████████████████████████████████████     | 476/507 [29:31<00:47,  1.54s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
 94%|███████████████████████████████████████████████████████████████████████████▍    | 478/507 [29:34<00:43,  1.50s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

 98%|██████████████████████████████████████████████████████████████████████████████  | 495/507 [30:12<00:37,  3.12s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tem_UTC['exstation_name'] = ex_station_en.inverse_transform(np.int8(y_data))
 98%|██████████████████████████████████████████████████████████████████████████████▎ | 496/507 [30:21<00:54,  4.92s/it]C:\Users\HLA_3D_DEV\AppData\Local\Temp\ipykernel_22656\1158696491.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/ind

In [43]:
out = pd.concat(out,ignore_index=True)

In [45]:
out.to_csv('0401XGBoost五.csv',index=False)